In [13]:
#pip install snowballstemmer
#pip install jpype1
#pip install zemberek-python
#pip install automata-lib

In [14]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from zemberek_python.main_libs import nltk_download, ZemberekTool
from kefir.predication import personal, inferential
from kefir.subject import Person
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')

In [15]:
from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)

In [16]:
morphology = TurkishMorphology.create_with_defaults()
normalizer = TurkishSentenceNormalizer(morphology)

2021-01-18 15:35:32,808 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 35.77569222450256



### Örnek bir cümle alıp üzerinde işlemlerimizi gerçekleştirelim.

In [20]:
text="ali okula gitti"

### Tokenleştirme işlemini gerçekleştirelim.

In [17]:
zemberekTool = ZemberekTool()

In [18]:
zemberekTool.ogelere_ayir("arabayla")

{'icerik': 'arabayla',
 'kok': 'araba',
 'tip': 'ISIM',
 'ekler': 'ISIM_KOK + ISIM_BIRLIKTELIK_LE'}

In [21]:
wordTokenize = word_tokenize(text)
wordTokenize

['ali', 'okula', 'gitti']

### Cümledeki kelimelerin ögelerini 'oge_bulucu' fonksiyonu yardımıyla öğrenelim

In [27]:
oge_list = []
for word in wordTokenize:
    oge_bulucu(word)
oge_list

['a', '.', 'a', 'e', '.', 'c', 'h', '.']

### Cümlenin kök ve ek bilgilerini otomatanın anlayacağı alfabeye dönüştürelim

In [25]:
ogeler =[]
ogeler2=""
for oge in oge_list:
    if oge == ".":
        ogeler.append(ogeler2)
        ogeler2=""
    else:
        ogeler2=ogeler2+oge
    print(ogeler," ",ogeler2)
ogeler

[]   a
['a']   
['a']   a
['a']   ae
['a', 'ae']   
['a', 'ae']   c
['a', 'ae']   ch
['a', 'ae', 'ch']   


['a', 'ae', 'ch']

### Cümlenin ögelerini sonlu durum otomatası yardımıyla bulalım

In [30]:
for i in ogeler:
    print(nfa.read_input(i))
print(oge_list,wordTokenize)

{'q_ozne'}
{'q_tumlec'}
{'q_yuklem'}
['a', '.', 'a', 'e', '.', 'c', 'h', '.'] ['ali', 'okula', 'gitti']


### Otomata diyagramı

<img src="otomata_diyagram.jpg">

### Tasarladığımız sonlu durum otomatası

In [8]:
dolaylı_tumlec_ekleri = ["ISIM_CIKMA_DEN","ISIM_KALMA_DE","ISIM_CIKMA_DE","ISIM_YONELME_E"]
from automata.fa.nfa import NFA
# NFA which matches strings beginning with 'a', ending with 'a', and containing
# no consecutive 'b's
nfa = NFA(
    states={'q_baslangic', 'q_ozne', 'q_zarf', 'q_yuklem', 'q_nesne', 'q_tumlec', 'q_edat'},
    input_symbols={'a', 'b', 'c', 'd' , 'e', 'f' ,'h'},
    transitions={
        'q_baslangic': {'a': {'q_ozne'}, 'b': {'q_zarf'}, 'c': {'q_yuklem'}},
        # Use '' as the key name for empty string (lambda/epsilon) transitions
        'q_ozne': {'d': {'q_nesne'}, 'e': {'q_tumlec'},'f': {'q_edat'}, 'h': {'q_ozne'}},
        'q_nesne': {'h': {'q_nesne'}},
        'q_tumlec': {'h': {'q_tumlec'}},
        'q_edat': {'h': {'q_edat'}},      
        'q_zarf': {'h': {'q_zarf'}},
        'q_yuklem': {'h': {'q_yuklem'}}

    },
    initial_state='q_baslangic',
    final_states={'q_ozne', 'q_zarf', 'q_yuklem', 'q_nesne', 'q_tumlec', 'q_edat'}
)

### Kelimelerin kök ve eklerinin türlerini bularak otomatanın alfabesine çeviren fonksiyon

In [9]:
oge_list = []
def oge_bulucu(word):
    if zemberekTool.ogelere_ayir(word) is None:
            oge_list.append("a")
            oge_list.append(".")
    else:         
        for ek_turu in zemberekTool.ogelere_ayir(word)["ekler"].replace(" ","").split("+"):

            if 'ISIM_KOK' == ek_turu:
                oge_list.append("a")
            elif 'ZAMAN_KOK' == ek_turu:
                oge_list.append("b")
            elif 'FIIL_KOK' == ek_turu:
                oge_list.append("c")
            elif 'ISIM_BELIRTME_I' == ek_turu or "ISIM_TAMLAMA_I" == ek_turu:
                oge_list.append("d")
            elif ek_turu in dolaylı_tumlec_ekleri:
                oge_list.append("e")
            elif 'ISIM_BIRLIKTELIK_LE' in zemberekTool.ogelere_ayir(word)["ekler"].replace(" ","").split("+"):
                oge_list.append("f")   

            else:
                oge_list.append("h")
            #[oge_list.append("e") if ek in dolaylı_tumlec_ekleri else oge_list.append("h") for ek in zemberekTool.ogelere_ayir(word)["ekler"].replace(" ","").split("+")]
        oge_list.append(".")